In [5]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from itertools import *
import seaborn as sns
from matplotlib.pyplot import figure

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt

#from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.feature_selection import mutual_info_regression

from sklearn.datasets import make_classification, make_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression
from scipy import stats
from sklearn.metrics import classification_report


## Датасеты и обработка

In [6]:

#titanic = pd.read_csv("/kaggle/input/test-file/tested.csv")
#titanic = titanic.drop(columns=['Name','Cabin','Ticket'])
#titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
#titanic['Fare'] = titanic['Fare'].fillna(titanic['Age'].median())
#titanic = pd.get_dummies(titanic, dtype = int)
#titanic = titanic[['Survived', 'Pclass', 'Age', 'SibSp', 'Fare']] 
pd.set_option('future.no_silent_downcasting', True)
income = pd.read_csv("/kaggle/input/income-classification/income_evaluation.csv")
income = income.replace({' <=50K':0,' >50K':1, ' Male':1, ' Female': 2})
income_tr = income[' income']
income_sex =  income[' sex']
income = income.drop(columns=[' native-country', ' income',  ' sex', ' marital-status'])
income['  workclass']= income[' workclass'].replace({' State-gov':'gov', ' Self-emp-not-inc':'Self-emp', ' Federal-gov':'gov',
       ' Local-gov':'gov', ' Self-emp-inc':'Self-emp', ' Without-pay':'no work',
       ' Never-worked':'no work'})

income[' education'] = income[' education'].replace(
{' Bachelors': 'high', ' HS-grad':'HS', ' 11th':'HS','11th':'HS', ' Masters':'high', ' 9th':'low',
       ' Some-college':'high', ' Assoc-acdm':'high', ' Assoc-voc':'high', ' 7th-8th':'low',
       ' Doctorate':'high', ' Prof-school':'high', ' 5th-6th': 'low', ' 10th':'high', ' 1st-4th':'low',
       ' Preschool':'low', ' 12th':'HS'})


income[' relationship'] = income[' relationship'].replace({' Not-in-family':'Unmarried', ' Husband':'married', ' Wife':'married', ' Own-child':'child', ' Unmarried':'Unmarried',
       ' Other-relative':'Other'})


income = pd.get_dummies(income)
income[' sex'] = income_sex
income[' income'] = income_tr

marketing = pd.read_csv("/kaggle/input/marketing-campaign-positive-response-prediction/campaign_responses.csv")
marketing = marketing.replace({'No':1, 'Yes':2, 'Male':1, 'Female':0, 'Married':1, 'Single':2})
student = pd.read_csv("/kaggle/input/student-habits-vs-academic-performance/student_habits_performance.csv")
student = student.replace({'No':1, 'Yes':2, 'Female':1, 'Male':2})
student['diet_quality'] = student['diet_quality'].replace({'Fair':2, 'Good':3, 'Poor':1})
student['parental_education_level'] = student['parental_education_level'].replace({'Master':3, 'High School':1, 'Bachelor':2})
student['internet_quality'] = student['internet_quality'].replace({'Average':2, 'Poor':1, 'Good':3})
student = student.dropna()
person = pd.read_csv("/kaggle/input/extrovert-vs-introvert-behavior-data/personality_datasert.csv")
person = person.replace({'No':1, 'Yes':2, 'Extrovert':1, 'Introvert':0})
heart = pd.read_csv("/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
heart2 = pd.read_csv("/kaggle/input/cardiovascular-disease-dataset/cardio_train.csv")
iris = pd.read_csv('/kaggle/input/iris/Iris.csv')
creditfraud = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
onlinefraud = pd.read_csv('/kaggle/input/online-payment-fraud-detection/onlinefraud.csv')
onlinefraud = onlinefraud.drop(columns=['nameOrig','nameDest','isFlaggedFraud']) 
bankruptcy = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')
b = bankruptcy['Bankrupt?']
bankruptcy = bankruptcy.drop(columns=['Bankrupt?']) 
bankruptcy['Bankrupt?'] = b

In [8]:

class Generator():

    def __init__(self, X, y, n_columns = 2):
       self.X = X
       self.y = y
       self.n_columns = n_columns # hiperparametr for generators witch split range of combination


    def split_list_by_chunks(self, data, n_columns):
        for i in range(0, len(data), self.n_columns ):
            yield data[i:i + self.n_columns]

    def multiply_gen(self, by=2, column_list=None): ## return np with annotation 
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in combinations(name_columns, by):
            combinations_list.append(name_col)
        
        combinations_list = self.split_list_by_chunks(combinations_list, self.n_columns )
        for seq in combinations_list:
            B = np.empty((len(seq),self.X.shape[0]))
            list_of_name = []
            for i, tup in enumerate(seq):
                A = np.ones(self.X.shape[0])
                list_of_name.append(f'multiply of - {" ".join(str(item) for item in tup)}')
                for col in tup:
                    A *= self.X[col].to_numpy()
                B[i] = A
            yield B, list_of_name
 

    def multiply(self, by=2, column_list=None, with_all_data=True):  ## return dataframe 
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in combinations(name_columns, by):
            combinations_list.append(name_col)      
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            A = np.ones(self.X.shape[0])
            list_of_name.append(f'multiply of - {" ".join(str(item) for item in tup)}')
            for col in tup:
                A *= self.X[col]
            B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
        

    def devision_on_last_gen(self, by=3, column_list=None):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, by):
            combinations_list.append(name_col)
        combinations_list = self.split_list_by_chunks(combinations_list, self.n_columns )

        for seq in combinations_list:
            B = np.empty((len(seq),self.X.shape[0]))
            list_of_name = [] #'multiply and devision - '
            for i, tup in enumerate(seq):
                A = np.ones(self.X.shape[0])   
                list_of_name.append(f'multiply on - {" ".join(str(item) for item in tup[:-1])}, and devision on - {tup[-1]}')
                for col in tup:
                    A *= self.X[col].to_numpy()
                    A = A/(self.X[tup[-1]]*self.X[tup[-1]])
                B[i] = A
            yield B, list_of_name


         

    def devision_on_last(self, by=3, column_list=None, with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, by):
            combinations_list.append(name_col)
            
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            A = np.ones(self.X.shape[0])
            list_of_name.append(f'multiply on - {" ".join(str(item) for item in tup[:-1])}, and devision on - {tup[-1]}')#.append(f'multiply of - {" ".join(str(item) for item in tup)}')
            for col in tup:
                A *= self.X[col]
                A = A/(self.X[tup[-1]]*self.X[tup[-1]])
                B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
    
            
    def devision_in_pairs(self, column_list=None, with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, 2):
            combinations_list.append(name_col)
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            list_of_name.append(f'{str(tup[0])} // {str(tup[1])}')
            A = X[tup[0]]/X[tup[1]]
            B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
            


    def log(self, column_list=None):
        name_columns = self.X.columns if column_list == None else column_list
        for col in name_columns:
            X['log '+ str(col)] = np.log(X[col])    
        df = X
        return df
        

    def exp(self, column_list=None):
        name_columns = self.X.columns if column_list == None else column_list
        for col in name_columns:
            X['exp '+ str(col)] = np.exp(X[col])    
        df = X
        return df

    def degree(self, by=2, column_list=None, with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, 2):
            combinations_list.append(name_col)
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            #print(tup[0])
            list_of_name.append(f'{str(tup[0])} ** {str(tup[1])}')
            A = X[tup[0]]**X[tup[1]]
            B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([self.X, self.y], axis=1)   
        else:     
            return df

    def devision_on_last_2(self, by=3, column_list=None,  with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in combinations(name_columns, by-1):
            combinations_list.append(name_col)
        A = len(combinations_list)*(self.X.shape[1]-2)

        
        B = np.empty((A, self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            A = np.ones(self.X.shape[0])
            dev_sub = [el for el in name_columns if el not in tup]

            for col in tup:
                A *= self.X[col]
                for dev in dev_sub:
                    list_of_name.append(f'multiply on - {" ".join(str(item) for item in tup)}, and devision on - {dev}')
                    A = A/self.X[dev]
                B[i] = A
        list_of_name  = list(dict.fromkeys(list_of_name))

        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df

    def deltas(self, column_list=None,  with_all_data=True):
        # use with standart scaler or normalize data only
        name_columns = self.X.columns if column_list == None else column_list
        combinations_list = []
        for name_col in combinations(name_columns, 2):
            combinations_list.append(name_col)
        list_of_name = []
        B = np.empty((len(combinations_list),self.X.shape[0]))
        for i, tup in enumerate(combinations_list): 
            list_of_name.append(f'{str(tup[0])} - {str(tup[1])}')
            B[i] = self.X[tup[0]]-self.X[tup[1]]
        df = pd.DataFrame(B.T, columns=list_of_name) #?????
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
        
    def sums():
        pass
  
        

class Correlations():
    def __init__(self, X, y):
        self.X = X #2d dimentional df
        self.y = y
    

    def mutual_info(self, mode='regression'):
        mi_dict = {}
        names_col = self.X.columns
        if mode == 'regression':
            v = mutual_info_regression(self.X, self.y)
        else:
             v = mutual_info_classif(self.X, self.y)
        
        for i, j in zip(names_col, v):
            mi_dict[i]=j

        mi_dict = dict(sorted(mi_dict.items(), key=lambda item: item[1]))
        return mi_dict


    def cramer_with_target(self):
        corr_val_crammer = {}
        for col in self.X.columns:
            confusion_matrix = pd.crosstab(self.X[col], self.y)
            chi2 = stats.chi2_contingency(confusion_matrix)[0]
            n = confusion_matrix.sum().sum()
            r, c = confusion_matrix.shape
            min_dim = min(r - 1, c - 1)
            v = np.sqrt(chi2 / (n * min_dim))
            corr_val_crammer['crammer with target ' + col]= v
        return corr_val_crammer


    def correlation(self, name='pearsonr'):
        corr_val = {}
        for col in self.X.columns:
            if name == 'pearsonr':
                res = stats.pearsonr(self.X[col], self.y)
            elif name == 'spearmanr': #Спирмен может быть полезна для нелинейных зависимостей.
                res = stats.spearmanr(self.X[col], self.y)
            elif name == 'kendalltau': #Кендалла, также используемой для ранговых данных
                res = stats.kendalltau(self.X[col], self.y)
            corr_val[col+' '+name] = res[0]
        return corr_val

    def heat_map(self):
        co_mtx = self.X.corr(numeric_only=True)
        w = co_mtx.shape[0]

        figure(figsize=(w, w), dpi=80)
        # Plot correlation heatmap
        sns.heatmap(co_mtx, cmap="YlGnBu", annot=True)



    def corr_feature_analisys(self):
        cor_dict = {}
        names = self.X.columns
        combinations_list = []
        for name_col in combinations(names, 2):
            combinations_list.append(name_col)

        for pairs in combinations_list:
            stat, _ = stats.pearsonr(self.X[pairs[0]], self.X[pairs[1]])
            cor_dict[pairs] = stat
        cor_dict = sorted(cor_dict.items(), key=lambda item: item[1], reverse=True)
        cor_dict = dict(cor_dict)

        return cor_dict

    def vis_corr_feature_analisys(self):
        cor_dict = self.corr_feature_analisys()
        keys = [' '.join(el) for el in cor_dict.keys()]
        #print(keys[:3], type(keys[2]))
        #for el in cor_dict.keys():
         #   cor_dict ', '.join(el)
        vals = [float(v) for v in cor_dict.values()]#[float(cor_dict.values()) for k in keys]
        #print(vals, keys)
        w = len(keys)

        figure(figsize=(w, w), dpi=80)
        sns.catplot(x=vals, y=keys)
  
        
        
    def report_cor():
        #p = correlation()
        #s = correlation()
        #k= correlation()
        pass

    def visulisation():
        pass

## Pipe for education and comparison

In [25]:
def train_test_scaller(df, name=''):
    
    y = df[name]
    X = df.drop([name], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=99)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def auto_trainer(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=13, max_depth=5, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    #y_pred_proba = clf.predict_proba(X_test[:2, :])
    clf.score(X_test, y_test)
   
    print('Report on test data')
    print(classification_report(y_test, y_pred))
    cf_matrix = confusion_matrix(y_test, y_pred)
    #plt.figure()
    #sns.heatmap(cf_matrix, annot=True)
    #$print(cf_matrix)
    
    print('Report on train data')
    print(classification_report(y_train, y_pred_train))
    cf_matrix = confusion_matrix(y_train, y_pred_train)
    print(cf_matrix)
    #plt.figure()
    #sns.heatmap(cf_matrix, annot=True)


#def feature_gen(df_inner, target_name, threshold=0.1): 
#    df = Generator(df_inner[df_inner.columns[:-1]], df_inner[target_name]).multiply(by=2) ## add some chose
#    corr = Correlations(df[df.columns[:-1]], df[target_name])
#    cor_dict = corr.corr_feature_analisys()
#    mi = corr.mutual_info()
#    mi_lim = {k:v for k, v in mi.items() if v < threshold} # exclude columns with have a mu info with target less than 0.1
#    mi_lim_ex = [col for col in mi_lim.keys() if col in df.columns]
#    df = df.drop(columns=mi_lim_ex, axis=1)
#    return df

#df_gen = Generator(heart[heart.columns[:-1]], heart['DEATH_EVENT']).multiply(by=2)
def feature_gen(df_gen, target_name, threshold=0.1): 
   ## add some chose
    corr = Correlations(df_gen[df_gen.columns[:-1]], df_gen[target_name])
    cor_dict = corr.corr_feature_analisys()
    mi = corr.mutual_info()
    mi_lim = {k:v for k, v in mi.items() if v < threshold} # exclude columns with have a mu info with target less than 0.1
    mi_lim_ex = [col for col in mi_lim.keys() if col in df_gen.columns]
    df_gen = df_gen.drop(columns=mi_lim_ex, axis=1)
    return df_gen

def simple_pipe(df_inner, df_gen, target_name, threshold):
    ndf = feature_gen(df_gen, target_name, threshold)
    X_train, X_test, y_train, y_test = train_test_scaller(df_inner, name=target_name)
    print('W/O manipulations')
    auto_trainer(X_train, X_test, y_train, y_test)
    print('With manipulations')
    X_train, X_test, y_train, y_test = train_test_scaller(ndf, name=target_name)
    auto_trainer(X_train, X_test, y_train, y_test)


In [27]:
df_gen = Generator(heart[heart.columns[:-1]], heart['DEATH_EVENT']).multiply(by=2)
simple_pipe(heart, df_gen, 'DEATH_EVENT', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        52
           1       0.74      0.74      0.74        23

    accuracy                           0.84        75
   macro avg       0.81      0.81      0.81        75
weighted avg       0.84      0.84      0.84        75

Report on train data
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       151
           1       0.97      0.85      0.91        73

    accuracy                           0.94       224
   macro avg       0.95      0.92      0.93       224
weighted avg       0.94      0.94      0.94       224

[[149   2]
 [ 11  62]]
With manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.87      0.90      0.89        52
           1       0.76      0.70      0.73        23

    accuracy                           0.84        75
   macr

In [33]:
df_gen = Generator(person[person.columns[:-1]], person['Personality']).multiply(by=2)
simple_pipe(person, df_gen, 'Personality', 0.1)

ValueError: data type <class 'numpy.object_'> not inexact

In [35]:
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Time_spent_Alone           2900 non-null   float64
 1   Stage_fear                 2900 non-null   object 
 2   Social_event_attendance    2900 non-null   float64
 3   Going_outside              2900 non-null   float64
 4   Drained_after_socializing  2900 non-null   object 
 5   Friends_circle_size        2900 non-null   float64
 6   Post_frequency             2900 non-null   float64
 7   Personality                2900 non-null   object 
dtypes: float64(5), object(3)
memory usage: 181.4+ KB


In [41]:
iris = iris.drop(columns='Id')

KeyError: "['Id'] not found in axis"

In [42]:
iris

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Fit and transform the 'Size' column
iris['Species'] = le.fit_transform(iris['Species'])

In [45]:
df_gen = Generator(iris[iris.columns[:-1]], iris['Species']).multiply(by=2)
simple_pipe(iris, df_gen, 'Species', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.93      0.93      0.93        14
           2       0.92      0.92      0.92        12

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Report on train data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        38

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.00       112
weighted avg       1.00      1.00      1.00       112

[[38  0  0]
 [ 0 36  0]
 [ 0  0 38]]
With manipulations
Report on test data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     

In [ ]:
df = feature_gen(iris, 'Personality')
X_train, X_test, y_train, y_test = train_test_scaller(df, name='Personality')
auto_trainer(X_train, X_test, y_train, y_test )

In [28]:
df = feature_gen(person, 'Personality')
X_train, X_test, y_train, y_test = train_test_scaller(df, name='Personality')
auto_trainer(X_train, X_test, y_train, y_test )

ValueError: data type <class 'numpy.object_'> not inexact